![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_Longformer.ipynb)

## Import Longformer models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 3.2.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for Longformer from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use Longformer models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.39.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 3.19.6 which is incompatible.
pandas

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [fxmarty/tiny-random-longformer-onnxtrue](https://huggingface.co/allenai/fxmarty/tiny-random-longformer-onnxtrue) model from HuggingFace as an example
- In addition to `TFLongformerModel` we also need to save the `LongformerTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [5]:
from transformers import LongformerTokenizer, TFLongformerModel
import tensorflow as tf

MODEL_NAME = 'fxmarty/tiny-random-longformer-onnxtrue'

# let's keep the tokenizer variable, we need it later
tokenizer = LongformerTokenizer.from_pretrained(MODEL_NAME)
# let's save the tokenizer
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFLongformerModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFLongformerModel.from_pretrained(MODEL_NAME, from_pt=True)

@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

try downloading TF weights


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

try downloading PyTorch weights


pytorch_model.bin:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFLongformerModel.

All the weights of TFLongformerModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {MODEL_NAME}

total 3504
-rw-r--r-- 1 root root     783 May 16 02:00 config.json
drwxr-xr-x 3 root root    4096 May 16 02:00 saved_model
-rw-r--r-- 1 root root 3577624 May 16 02:00 tf_model.h5


In [7]:
!ls -l {MODEL_NAME}/saved_model/1

total 12164
drwxr-xr-x 2 root root     4096 May 16 02:00 assets
-rw-r--r-- 1 root root       53 May 16 02:00 fingerprint.pb
-rw-r--r-- 1 root root    43109 May 16 02:00 keras_metadata.pb
-rw-r--r-- 1 root root 12395300 May 16 02:00 saved_model.pb
drwxr-xr-x 2 root root     4096 May 16 02:00 variables


In [8]:
!ls -l {MODEL_NAME}_tokenizer

total 1432
-rw-r--r-- 1 root root 456318 May 16 01:59 merges.txt
-rw-r--r-- 1 root root    957 May 16 01:59 special_tokens_map.json
-rw-r--r-- 1 root root   1194 May 16 01:59 tokenizer_config.json
-rw-r--r-- 1 root root 999355 May 16 01:59 vocab.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.json` and `merges.txt` files from the tokenizer
- all we need is to first convert `vocab.json` to `vocab.txt` and copy both `vocab.txt` and `merges.txt` into `saved_model/1/assets` which Spark NLP will look for

In [9]:
# let's save the vocab as txt file
with open('{}_tokenizer/vocab.txt'.format(MODEL_NAME), 'w') as f:
    for item in tokenizer.get_vocab().keys():
        f.write("%s\n" % item)

# let's copy both vocab.txt and merges.txt files to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/merges.txt {MODEL_NAME}/saved_model/1/assets

## Import and Save Longformer in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [10]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-05-16 02:00:55--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-05-16 02:00:55--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-05-16 02:00:55 (60.5 MB/s) - written to stdout [1191/1191]

Installing

Let's start Spark with Spark NLP included via our simple `start()` function

In [11]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


- Let's use `loadSavedModel` functon in `LongformerEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `LongformerEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` only accepts local paths and not distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. That is why we use `write.save` so we can use `.load()` from any file systems.



In [12]:
from sparknlp.annotator import *

longformer = LongformerEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("embeddings")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setMaxSentenceLength(4096)\
 .setStorageRef('longformer_base_4096')


- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [13]:
longformer.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [14]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your Longformer model from HuggingFace 🤗 loaded and saved by Spark NLP 🚀

In [15]:
! ls -l {MODEL_NAME}_spark_nlp

total 15488
drwxr-xr-x 5 root root     4096 May 16 02:03 fields
-rw-r--r-- 1 root root 15850073 May 16 02:03 longformer_tensorflow
drwxr-xr-x 2 root root     4096 May 16 02:03 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny Longformer model 😊

In [16]:
longformer_loaded = LongformerEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("embeddings")

In [17]:
longformer_loaded.getStorageRef()

'longformer_base_4096'

That's it! You can now go wild and use hundreds of Longformer models from HuggingFace 🤗 in Spark NLP 🚀
